In [3]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from VAD_util import calculate_lambda, prediction_transformation

# fix random seed so it's easier to reproduce the result
np.random.seed(1234)

num_features = 20
sigma = 0.1
num_examples = 30000
num_examples_train = 3000
num_simulation = 100
S_GROUP = 2

result_keys = [
    'total_calibration_p',
    'total_calibration_y',
    'positive_ratio_after_selection',
    'positive_ratio_train',
    'positive_ratio_test',
    'Ground truth',
    'calibration_y_true_p',
    'calibration_p_true_p',
    'calibration_p_y_true_p',
    'Vanilla',
    'Vanilla_p',
    'Vanilla_ECE',
    'Vanilla_MCE',
    'VAD',
    'VAD_p',
    'VAD_ECE',
    'VAD_MCE',
    'VAD prob',
    'VAD prob_p',
    'VAD prob_ECE',
    'VAD prob_MCE',
    'Lambda logit',
    'Lambda prob',
    'Mu logit',
    'Mu prob',
    'Log Loss Logit Improvement',
    'Log Loss Prob Improvement',
]

def generate_data(num_examples, mu):
    x = np.random.normal(mu, sigma, (num_examples, num_features))
    p = 1 / (1 + np.exp(- np.sum(x, axis=1)))
    y = np.random.binomial(1, p)
    return x, y, p

def bootstrap_data(x, y):
    num_examples = x.shape[0]
    ind = np.random.choice(np.arange(num_examples), num_examples, replace=True)
    return x[ind], y[ind]

def calculate_ECE_MCE_Brier(y_prob, y_true, n_bins):
    sorted_indices = np.argsort(y_prob)
    sorted_y_true = y_true[sorted_indices]
    sorted_y_prob = y_prob[sorted_indices]
    binned_y_true = np.array_split(sorted_y_true, n_bins)
    binned_y_prob = np.array_split(sorted_y_prob, n_bins)
    ece_errors = 0.0
    mce_errors = 0.0
    brier_errors = 0.0
    for bin_y_true, bin_y_prob in zip(binned_y_true, binned_y_prob):
        avg_y_true = np.mean(bin_y_true)
        avg_y_score = np.mean(bin_y_prob)
        ce_error = np.abs(avg_y_score - avg_y_true)
        ece_errors += ce_error / n_bins
        mce_errors = max(mce_errors, ce_error)
        brier_errors += ((avg_y_score - avg_y_true) ** 2) / n_bins
    return round(ece_errors, 4), round(mce_errors, 4), round(brier_errors, 4)

def VAD_method(num_simulation, num_group, method, alpha, train_data, test_data, test_val_data):

    def calculate_logit(X, clf):
        return np.squeeze(X @ clf.coef_.T + clf.intercept_)

    def append_result(result, curr):
        for result_key in result_keys:
            result[result_key].append(curr[result_key])
        return result

    assert method in ['bootstrap', 'seperate_data']
    x_train_array, y_train_array, p_train_array = train_data
    x_test_val_array, _, _ = test_val_data 
    x_test_array, y_test_array, p_test_array = test_data 

    result = {}
    for result_key in result_keys:
        result[result_key] = []

    for k in range(num_simulation):
        x_train, y_train, p_train = x_train_array[k], y_train_array[k], p_train_array[k]
        x_test, y_test, p_test = x_test_array[k], y_test_array[k], p_test_array[k]
        x_test_val = x_test_val_array[k]
        clf = LogisticRegression(penalty='none', random_state=0).fit(x_train, y_train)
        p_predicted = clf.predict_proba(x_test)[:, 1]
        p_predicted_test_val = clf.predict_proba(x_test_val)[:, 1]
        logit_predicted = calculate_logit(x_test, clf)
        logit_predicted_test_val = calculate_logit(x_test_val, clf)
        num_examples_test_val = x_test_val.shape[0]
        num_examples_test = x_test.shape[0]
        p_predicted_subgroup = np.zeros((num_examples_test_val, num_group))
        logit_predicted_subgroup = np.zeros((num_examples_test_val, num_group))
        for i in range(num_group):
            x_train_bootstrap, y_train_bootstrap = bootstrap_data(x_train, y_train)
            clf_i = LogisticRegression(penalty='none', random_state=0).fit(x_train_bootstrap, y_train_bootstrap)
            p_predicted_subgroup[:, i] = clf_i.predict_proba(x_test_val)[:, 1]
            logit_predicted_subgroup[:, i] = calculate_logit(x_test_val, clf_i)

        lambda_p_logit = calculate_lambda(p_predicted_subgroup, p_predicted_test_val, 'logit', logit_predicted_subgroup, logit_predicted_test_val)
        lambda_p_prob = calculate_lambda(p_predicted_subgroup, p_predicted_test_val, 'probability', logit_predicted_subgroup, logit_predicted_test_val)

        choose_num_examples = int(num_examples_test * alpha)
        ind = np.argpartition(p_predicted, -choose_num_examples)[-choose_num_examples:]
        ind_true_p =  np.argpartition(p_test, -choose_num_examples)[-choose_num_examples:]

        log_loss = np.sum(-(y_test[ind] * np.log(p_predicted[ind]) + (1 - y_test[ind]) * np.log(1 - p_predicted[ind])))
        refined_prediction_logit = prediction_transformation(p_predicted, ind, lambda_p_logit, 'logit', np.mean(p_predicted_test_val), np.mean(logit_predicted_test_val), logit_predicted)
        refined_prediction_prob = prediction_transformation(p_predicted, ind, lambda_p_prob, 'probability', np.mean(p_predicted_test_val), np.mean(logit_predicted_test_val), logit_predicted)
        refined_log_loss_logit = np.sum(-(y_test[ind] * np.log(refined_prediction_logit) + (1 - y_test[ind]) * np.log(1 - refined_prediction_logit)))
        refined_log_loss_prob = np.sum(-(y_test[ind] * np.log(refined_prediction_prob) + (1 - y_test[ind]) * np.log(1 - refined_prediction_prob)))

        ECE, MCE, Brier = calculate_ECE_MCE_Brier(p_predicted[ind], y_test[ind], 10)
        ECE_logit, MCE_logit, Brier_logit = calculate_ECE_MCE_Brier(refined_prediction_logit, y_test[ind], 10)
        ECE_prob, MCE_prob, Brier_prob = calculate_ECE_MCE_Brier(refined_prediction_prob, y_test[ind], 10)

        curr = {
            'total_calibration_p': np.sum(p_predicted) / np.sum(p_test),
            'total_calibration_y': np.sum(p_predicted) / np.sum(y_test),
            'positive_ratio_after_selection': np.sum(y_test[ind]) / len(y_test[ind]),
            'positive_ratio_train': np.sum(y_train) / len(y_train),
            'positive_ratio_test': np.sum(y_test) / len(y_test),
            'Ground truth': np.sum(p_test[ind]) / np.sum(y_test[ind]),
            'calibration_y_true_p': np.sum(p_predicted[ind_true_p]) / np.sum(y_test[ind_true_p]),
            'calibration_p_true_p': np.sum(p_predicted[ind_true_p]) / np.sum(p_test[ind_true_p]),
            'calibration_p_y_true_p': np.sum(y_test[ind_true_p]) / np.sum(p_test[ind_true_p]),
            'Vanilla': np.sum(p_predicted[ind]) / np.sum(y_test[ind]),
            'Vanilla_p': np.sum(p_predicted[ind]) / np.sum(p_test[ind]),
            'Vanilla_ECE': ECE,
            'Vanilla_MCE': MCE,
            'VAD': np.sum(refined_prediction_logit) / np.sum(y_test[ind]),
            'VAD_p': np.sum(refined_prediction_logit) / np.sum(p_test[ind]),
            'VAD_ECE': ECE_logit,
            'VAD_MCE': MCE_logit,
            'VAD prob': np.sum(refined_prediction_prob) / np.sum(y_test[ind]),
            'VAD prob_p': np.sum(refined_prediction_prob) / np.sum(p_test[ind]),
            'VAD prob_ECE': ECE_prob,
            'VAD prob_MCE': MCE_prob,
            'Lambda logit': lambda_p_logit,
            'Lambda prob': lambda_p_prob,
            'Mu logit': np.mean(logit_predicted_test_val),
            'Mu prob': np.mean(p_predicted_test_val),
            'Log Loss Logit Improvement': (refined_log_loss_logit - log_loss) / log_loss * 100,
            'Log Loss Prob Improvement': (refined_log_loss_prob - log_loss) / log_loss * 100,
        }
        result = append_result(result, curr)

    print("alpha: ", alpha)
    for result_key in result_keys:
        result[result_key] = np.array(result[result_key])
        print("mean of", result_key, ": ", np.mean(result[result_key]))
    print()

    return result

def generate_data_array(num_simulation, num_examples, mu):
    x_array = []
    y_array = []
    p_array = []
    for k in range(num_simulation):
        x, y, p = generate_data(num_examples, mu)
        x_array.append(x)
        y_array.append(y)
        p_array.append(p)
    return (x_array, y_array, p_array)


mu = -0.05
mu_train = 0.05
train_data = generate_data_array(num_simulation, num_examples_train, mu_train)
test_data = generate_data_array(num_simulation, num_examples, mu)
test_val_data = generate_data_array(num_simulation, num_examples, mu)

report = {}
reported_result_keys = [
    'Vanilla',
    'Vanilla_ECE',
    'Vanilla_MCE',
    'VAD',
    'VAD_ECE',
    'VAD_MCE',
    'VAD prob',
    'VAD prob_ECE',
    'VAD prob_MCE',
]

alphas = [0.02, 0.1]
for alpha in alphas:
    result = VAD_method(num_simulation, S_GROUP, 'bootstrap', alpha, train_data, test_data, test_val_data)
    alpha_key = int(alpha * 1000)
    report[alpha_key] = {}
    for result_key in reported_result_keys:
        curr_result = result[result_key]
        total_num_result = curr_result.shape[0]
        report[alpha_key][result_key] = (np.mean(curr_result), np.std(curr_result) / np.sqrt(total_num_result))

alpha:  0.02
mean of total_calibration_p :  1.0075426623474724
mean of total_calibration_y :  1.00684832888181
mean of positive_ratio_after_selection :  0.5006666666666666
mean of positive_ratio_train :  0.7223800000000001
mean of positive_ratio_test :  0.27763566666666667
mean of Ground truth :  1.0014845514782749
mean of calibration_y_true_p :  1.0017355436161197
mean of calibration_p_true_p :  1.0008945827106528
mean of calibration_p_y_true_p :  1.000854484927436
mean of Vanilla :  1.0870833403158637
mean of Vanilla_p :  1.085150644350889
mean of Vanilla_ECE :  0.06580099999999998
mean of Vanilla_MCE :  0.14744999999999997
mean of VAD :  0.9947581526963414
mean of VAD_p :  0.9930470981433029
mean of VAD_ECE :  0.05720600000000001
mean of VAD_MCE :  0.12937900000000002
mean of VAD prob :  1.0053367056113378
mean of VAD prob_p :  1.0035978541514707
mean of VAD prob_ECE :  0.05689999999999999
mean of VAD prob_MCE :  0.129316
mean of Lambda logit :  0.8393979321066222
mean of Lambda pro